In [6]:
from dataclasses import dataclass

from pulp import LpInteger, LpMaximize, LpMinimize, LpProblem, LpStatus, LpVariable, lpSum, value


## Ilość osób danego dnia:
- Poniedziałek : 2
- Wtorek : 1
- Środa : 1
- Czwartek : 1
- Piątek : 3

## Oczekiwania osób:
- Ania : 100 zł/h \[Pon-Pt]
- Stefan : 50 zł/h \[Pon]
- Hektor : 60 zł/h \[Pon-Śr]
- Olaf : 40 zł/h \[Pon-Pt]
- Lidia : 110 zł/h \[Czw-Pt]
- Piotr : 70 zł/h \[Pon-Śr]


In [5]:
week_days: list[str] = ["PON", "WT", "SR", "CZW", "PT"]
workers: list[str] = ["Ania", "Stefan", "Hektor", "Olaf", "Lidia", "Piotr"]


workers_per_day: dict[str, int] = {
    week_days[0]: 2,
    week_days[1]: 1,
    week_days[2]: 1,
    week_days[3]: 1,
    week_days[4]: 3,
}

payment_per_worker: dict[str, int] = {
    workers[0]: 100,
    workers[1]: 50,
    workers[2]: 60,
    workers[3]: 40,
    workers[4]: 110,
    workers[5]: 70,
}


@dataclass
class Worker:
    def __init__(self, name: str, available_days: list[int] = [0, 0, 0, 0, 0]):
        self.name = name
        self.available_days = available_days


worker_0: Worker = Worker(workers[0], [1, 1, 1, 1, 1])
worker_1: Worker = Worker(workers[1], [1, 0, 0, 0, 0])
worker_2: Worker = Worker(workers[2], [1, 1, 1, 0, 0])
worker_3: Worker = Worker(workers[3], [1, 1, 1, 1, 1])
worker_4: Worker = Worker(workers[4], [0, 0, 0, 1, 1])
worker_5: Worker = Worker(workers[5], [1, 1, 1, 0, 0])


problem = LpProblem("Scheduling_problem", LpMinimize)


worker_0_var: LpVariable = LpVariable.dicts(f"{worker_0.name}", week_days, 0, 1, LpInteger)
worker_1_var: LpVariable = LpVariable.dicts(f"{worker_1.name}", week_days, 0, 1, LpInteger)
worker_2_var: LpVariable = LpVariable.dicts(f"{worker_2.name}", week_days, 0, 1, LpInteger)
worker_3_var: LpVariable = LpVariable.dicts(f"{worker_3.name}", week_days, 0, 1, LpInteger)
worker_4_var: LpVariable = LpVariable.dicts(f"{worker_4.name}", week_days, 0, 1, LpInteger)
worker_5_var: LpVariable = LpVariable.dicts(f"{worker_5.name}", week_days, 0, 1, LpInteger)

problem += (
    4
    * lpSum(
        payment_per_worker[workers[0]] * [worker_0_var[i] for i in week_days]
        + payment_per_worker[workers[1]] * [worker_1_var[i] for i in week_days]
        + payment_per_worker[workers[2]] * [worker_2_var[i] for i in week_days]
        + payment_per_worker[workers[3]] * [worker_3_var[i] for i in week_days]
        + payment_per_worker[workers[4]] * [worker_4_var[i] for i in week_days]
        + payment_per_worker[workers[5]] * [worker_5_var[i] for i in week_days]
    ),
    "Payment",
)

for i in week_days:
    problem += worker_0_var[i] + worker_1_var[i] + worker_2_var[i] + worker_3_var[i] + worker_4_var[i] + worker_5_var[i] == workers_per_day[i]


for i, x in enumerate(week_days):
    problem += worker_0_var[x] <= worker_0.available_days[i], f"{workers[0]}_available: " + str(i)
    problem += worker_1_var[x] <= worker_1.available_days[i], f"{workers[1]}_available: " + str(i)
    problem += worker_2_var[x] <= worker_2.available_days[i], f"{workers[2]}_available: " + str(i)
    problem += worker_3_var[x] <= worker_3.available_days[i], f"{workers[3]}_available: " + str(i)
    problem += worker_4_var[x] <= worker_4.available_days[i], f"{workers[4]}_available: " + str(i)
    problem += worker_5_var[x] <= worker_5.available_days[i], f"{workers[5]}_available: " + str(i)

problem += lpSum(worker_0_var[i] for i in week_days) <= 3, f"{workers[0]}_workday_limit: " + str(i)
problem += lpSum(worker_3_var[i] for i in week_days) <= 3, f"{workers[3]}_workday_limit: " + str(i)

problem.solve()

final_payment: dict[str, int] = {
    workers[0]: 0,
    workers[1]: 0,
    workers[2]: 0,
    workers[3]: 0,
    workers[4]: 0,
    workers[5]: 0,
}


for v in problem.variables():
    if v.varValue == 1:
        for worker in workers:
            if worker in v.name:
                final_payment[worker] += payment_per_worker[worker] * 8
        print(v.name, "=", v.varValue)

for i in final_payment:
    print(f"{i}: {final_payment[i]} zł")

print(f"Weekly expenses: {sum(final_payment.values())} zł")

Ania_PT = 1.0
Hektor_PON = 1.0
Hektor_SR = 1.0
Lidia_PT = 1.0
Olaf_CZW = 1.0
Olaf_PT = 1.0
Olaf_WT = 1.0
Stefan_PON = 1.0
Ania: 800 zł
Stefan: 400 zł
Hektor: 960 zł
Olaf: 960 zł
Lidia: 880 zł
Piotr: 0 zł
Weekly expenses: 4000 zł
